In [15]:
import tensorflow as tf
import numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt

# Dropout

Aby zaimplementować **dropout** za pomocą TensorFlow, możesz po prostu zastosować funkcję 
```python
tf.layers.dropout 
```
do warstwy wejściowej.

In [16]:
tf.reset_default_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [17]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [18]:
training = tf.placeholder_with_default(False, shape=(), name='training')

dropout_rate = 0.5  # == 1 - keep_prob
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, activation=tf.nn.relu,
                              name="hidden1")
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation=tf.nn.relu,
                              name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")

In [19]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()

In [20]:
n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(mnist.train.num_examples // batch_size):
            print("\r{}%".format(100 * batch_index //  (mnist.train.num_examples // batch_size) ), end="")
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(" ", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test) 

99%  0 Train accuracy: 0.925 Test accuracy: 0.9182
99%  1 Train accuracy: 0.94 Test accuracy: 0.9409
99%  2 Train accuracy: 0.945 Test accuracy: 0.9493
99%  3 Train accuracy: 0.97 Test accuracy: 0.9575
99%  4 Train accuracy: 0.97 Test accuracy: 0.9595
99%  5 Train accuracy: 0.985 Test accuracy: 0.9651
99%  6 Train accuracy: 0.965 Test accuracy: 0.9674
99%  7 Train accuracy: 0.98 Test accuracy: 0.97
99%  8 Train accuracy: 0.98 Test accuracy: 0.9701
99%  9 Train accuracy: 0.965 Test accuracy: 0.9715
99%  10 Train accuracy: 0.99 Test accuracy: 0.9742
99%  11 Train accuracy: 0.995 Test accuracy: 0.9752
99%  12 Train accuracy: 0.99 Test accuracy: 0.9755
99%  13 Train accuracy: 0.995 Test accuracy: 0.9782
99%  14 Train accuracy: 1.0 Test accuracy: 0.9763
99%  15 Train accuracy: 1.0 Test accuracy: 0.9753
99%  16 Train accuracy: 0.99 Test accuracy: 0.9776
99%  17 Train accuracy: 0.985 Test accuracy: 0.9787
99%  18 Train accuracy: 0.99 Test accuracy: 0.9783
99%  19 Train accuracy: 0.985 Test ac